In [2]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00


In [3]:
!pip install faiss-cpu sentence-transformers langchain langchain-openai --quiet

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from langchain.docstore.document import Document

In [6]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter

transcript_folder = "/content/drive/MyDrive/ServiceNow_Audio_Transcripts/"
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

documents = []

for filename in os.listdir(transcript_folder):
    if filename.endswith(".txt"):
        path = os.path.join(transcript_folder, filename)
        with open(path, "r", encoding="utf-8") as f:
            text = f.read()
            chunks = text_splitter.split_text(text)
            for chunk in chunks:
                documents.append(Document(page_content=chunk, metadata={"source": filename}))

In [7]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = FAISS.from_documents(documents, embedding=embedding_model)

/tmp/ipython-input-7-3026385982.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
from google.colab import userdata
llm = OpenAI(
    temperature=0,
    openai_api_key=userdata.get("OPENAI_API_KEY")
)

In [9]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

In [10]:
question = "How does ServiceNow use AI in incident management?"
result = qa_chain(question)

print("🧠 Answer:\n", result["result"])
print("\n📚 Sources:")
for doc in result["source_documents"]:
    print("-", doc.metadata["source"])

/tmp/ipython-input-10-1475440182.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain(question)


🧠 Answer:
  ServiceNow uses AI in incident management through their AI Ops solution, which utilizes machine learning and pattern recognition to assist and automate the work of IT operations. This helps move away from an incident creation culture to an alert-based, actionable mindset. The AI is built into the ServiceNow platform and is not acquired from other companies, making it a native and additive part of the platform's innovations and investments.

📚 Sources:
- kQV6g8Vbbfc.txt
- KSWNDuKn9t0.txt
- WyQTP0AA1VU.txt
- fqB-NcZmqXo.txt


In [11]:
question = "What is CMDB in ServiceNow?"
result = qa_chain(question)

print("🧠 Answer:\n", result["result"])
print("\n📚 Sources:")
for doc in result["source_documents"]:
    print("-", doc.metadata["source"])

🧠 Answer:
  CMDB stands for Configuration Management Database and it is a feature in ServiceNow that allows users to track and manage all of the assets and configuration items in their IT environment. It is a central repository for all IT assets and their relationships, providing a comprehensive view of the entire IT infrastructure.

📚 Sources:
- K6z4c256gzI.txt
- kQV6g8Vbbfc.txt
- K6z4c256gzI.txt
- kQV6g8Vbbfc.txt


In [12]:
question = "How is AI used in ServiceNow Project Management?"
result = qa_chain(question)

print("🧠 Answer:\n", result["result"])
print("\n📚 Sources:")
for doc in result["source_documents"]:
    print("-", doc.metadata["source"])

🧠 Answer:
  AI is used in ServiceNow Project Management through the AI Ops solution, which assists and automates the work of IT operations using AI fundamentals like machine learning and pattern recognition. It also helps optimize the value of AI across the organization and can be integrated with other AI tools from companies like Google and Microsoft. Additionally, AI is used in Strategic Portfolio Management to create assisted and autonomous workflows.

📚 Sources:
- KSWNDuKn9t0.txt
- eFMeZto6yMg.txt
- kQV6g8Vbbfc.txt
- mSYdZW_D67o.txt
